In [3]:
import pydicom as dcm

In [8]:
dcmds = dcm.read_file('RS.1.2.246.352.205.4955711944111358107.16273022557623532164.dcm')
strsetsq = dcmds.StructureSetROISequence

In [15]:
strnames = [structure.ROIName for structure in strsetsq]

In [19]:
for strname in strnames:
    if len(strname) > 16:
        print(strname)

In [20]:
strnames

['BODY',
 'Ao',
 'Arteria Pulmonar',
 'Aurícula dcha',
 'Aurícula izqda',
 'Corazón',
 'CTV boost izqdo',
 'CTV mama izqda',
 'Esófago',
 'Mama dcha',
 'Médula',
 'PRV DA',
 'PTV boost izqdo',
 'PTV mama izqda',
 'Pulmón dcho',
 'Pulmón izqdo',
 'Tiroides',
 'Vena cava inf',
 'Vena cava sup',
 'Ventrículo dcho',
 'Ventrículo izqdo']

In [21]:
from textdistance import ratcliff_obershelp

In [30]:
{strname: ratcliff_obershelp('aurícula derecha', strname) for strname in strnames}

{'BODY': 0.0,
 'Ao': 0.0,
 'Arteria Pulmonar': 0.3125,
 'Aurícula dcha': 0.8275862068965517,
 'Aurícula izqda': 0.6666666666666666,
 'Corazón': 0.08695652173913043,
 'CTV boost izqdo': 0.12903225806451613,
 'CTV mama izqda': 0.3333333333333333,
 'Esófago': 0.08695652173913043,
 'Mama dcha': 0.56,
 'Médula': 0.2727272727272727,
 'PRV DA': 0.09090909090909091,
 'PTV boost izqdo': 0.12903225806451613,
 'PTV mama izqda': 0.3333333333333333,
 'Pulmón dcho': 0.4444444444444444,
 'Pulmón izqdo': 0.2857142857142857,
 'Tiroides': 0.25,
 'Vena cava inf': 0.27586206896551724,
 'Vena cava sup': 0.27586206896551724,
 'Ventrículo dcho': 0.5806451612903226,
 'Ventrículo izqdo': 0.4375}